## AutoQuake Demo - Complete Earthquake Catalog Generation
---
***Welcome to the AutoQuake demonstration!***     
***This notebook will guide you through the complete earthquake catalog generation workflow:***

1. **Download sample seismic data**

2. **Phase Detection** with `PhaseNet`
3. **Event Association** with `GaMMA` 
4. **Relocation** with `H3DD`
5. **Magnitude Estimation** with CWA standard
6. **Polarity Determination** with `DiTingMotion`
7. **Focal Mechanism** with `GAfocal`

### ***Prerequisites***
- Install the conda environment (`conda env create -f env.yml`), and select it for kernel

- Internet connection for downloading demo data
- Compile the H3DD and GAFocal

---

### ***Step 1: Download Demo Data and Setup***

First, let's download sample seismic data and import necessary modules.

In [1]:
import os
import shutil
import sys
import tarfile
import requests
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# Add AutoQuake to path
sys.path.append('.')

# Import AutoQuake components (following predict.py structure)
from autoquake.picker import PhaseNet, run_predict
from autoquake.associator import GaMMA
from autoquake.focal import GAfocal
from autoquake.magnitude import Magnitude
from autoquake.polarity import DitingMotion
from autoquake.relocator import H3DD
from autoquake.utils import pol_mag_to_dout

from ParamConfig.config_model import PhaseNetConfigReceiver

print("All AutoQuake components imported successfully!")

All AutoQuake components imported successfully!


In [16]:

# Demo Configuration - Set your parameters here!
record_id = "17470233"
filename = "toy_data.tgz"

DEMO_DATA_URL = f"https://zenodo.org/records/{record_id}/files/{filename}?download=1"
DEMO_DATA_DIR = Path("./toy_data")
DEMO_DATA_DIR.mkdir(parents=True, exist_ok=True)

try:
    response = requests.get(DEMO_DATA_URL, stream=True, timeout=60)
    with open(DEMO_DATA_DIR / filename, 'wb') as f:
        f.write(response.content)
    print("Downloaded!")

    with tarfile.open(DEMO_DATA_DIR / filename, 'r:gz') as tar:
        # Get all members and strip the first directory level
        for member in tar.getmembers():
            # Skip the root 'toy_data/' directory itself
            if member.name == 'toy_data' or member.name == 'toy_data/':
                continue            
            if member.name.startswith('toy_data/'):
                member.name = member.name.replace('toy_data/', '', 1)  # Remove first 'toy_data/'
            tar.extract(member, DEMO_DATA_DIR)
    print(f"Demo data extracted to '{DEMO_DATA_DIR}'")
except Exception as e:
    print(f"Error: {e}")
    print("Please download manually from:", DEMO_DATA_URL)


Downloaded!
Demo data extracted to 'toy_data'


In [19]:
NCPU = 2

RESULT_DIR = Path("demo_results")
RESULT_DIR.mkdir(exist_ok=True)

STATION = DEMO_DATA_DIR / "station.csv"
SAC_PARENT_DIR = DEMO_DATA_DIR / "data"
PZ_DIR = DEMO_DATA_DIR / "PZ"
H3DD_VEL_FILE = Path("./H3DD/tomops_H14")

### ***Step 2: Phase Detection with PhaseNet***

PhaseNet is a deep learning model that automatically detects P and S wave arrivals in seismic data. This is the first step in earthquake catalog generation.

In [ ]:
phasenet_config = PhaseNetConfigReceiver(
    data_parent_dir=SAC_PARENT_DIR,
    start="20240402",
    end="20240404",
    result_path=RESULT_DIR,
    pz_dir=PZ_DIR,
    min_prob=0.5  
)

run_predict(phasenet_config.args_list)

phase_picks = PhaseNet.concat_picks(
    date_list=['20240402', '20240403'],
    result_path=RESULT_DIR,
)

### ***Step 3: Event Association with GaMMA***

GaMMA (Gaussian Mixture Model Associator) groups individual phase picks into earthquake events using machine learning clustering techniques.

In [5]:
def phasenet_station_name_processing(pickings: Path, output_dir: Path) -> Path:
    shutil.copy(pickings, output_dir/f"{pickings.stem}_original.csv")
    df = pd.read_csv(pickings)
    df['station_id'] = df['station_id'].map(lambda x: str(x).split('.')[1])
    output_path = output_dir/pickings.name
    df.to_csv(output_path, index=False)
    return output_path

# Preprocessing for GaMMA (as in predict.py)
post_phasenet_pickings = phasenet_station_name_processing(
    pickings=Path("/home/patrick/Work/AutoQuake/demo_results/picks_phasenet/picks.csv"),
    output_dir=RESULT_DIR
)

# Initialize GaMMA with direct parameters
# choose number of processes: at most half of available CPUs, but not exceeding configured ncpu
avail_cpus = os.cpu_count() or NCPU
ncpu_to_use = min(max(1, avail_cpus // 2), NCPU)

gamma = GaMMA(
    station=STATION,
    result_path=RESULT_DIR,
    center=(121.625, 24.0),
    xlim_degree=[121.0, 122.25],
    ylim_degree=[23.25, 24.75],
    pickings=post_phasenet_pickings,
    min_p_picks_per_eq=6,
    min_s_picks_per_eq=2,
    dbscan_eps=17.33,
    ncpu=ncpu_to_use,
    use_amplitude=False
)

# Run event association
gamma.run_predict()

Splitting 2550 picks using eps=14.44
Splitting 2086 picks using eps=14.44
Splitting 1382 picks using eps=14.44
Splitting 2246 picks using eps=12.03
Splitting 2082 picks using eps=12.03
Associating 135 clusters with 40 CPUs
.......................................................................................................................................
Associated 100 events

Associated 200 events


### ***Step 4: Relocation with H3DD***

Relocation with Hypo-DD strengthen by 3D velocity models.

In [ ]:
print("Running H3DD...")
h3dd = H3DD(
    gamma_event=gamma.get_events(),
    gamma_picks=gamma.get_picks(),
    result_path=RESULT_DIR,
    station=STATION,
    model_3d=H3DD_VEL_FILE,
    event_name="demo_c0",
    cut_off_distance_for_dd=0.0
)
h3dd.run_h3dd()

### ***Step 5: Magnitude Estimation***

Calculate earthquake magnitudes with **regional specific magnitude empirical formula**.

In [ ]:
mag = Magnitude(
    dout_file=h3dd.get_dout(),
    station=STATION,
    sac_parent_dir=SAC_PARENT_DIR,
    pz_dir=PZ_DIR,
    output_dir=RESULT_DIR,
)
mag.run_mag(processes=2)  # Use 2 processes for demo

### ***Step 6: DiTingMotion - Polarity determination***

Analyze first-motion polarities for focal mechanism determination.

In [40]:
def type_judge(station: str):
    """
    This is designed for determining station type based on its name.
    Typically, DAS stations start with a number, while traditional seismic stations start with a letter.
    """
    return station[0].isalpha()

dt_polarity = DitingMotion(
    gamma_picks=gamma.get_picks(),
    output_dir=RESULT_DIR,
    sac_parent_dir=SAC_PARENT_DIR,
    type_judge=type_judge
)
dt_polarity.run_parallel_predict(processes=2)

### ***Step 7: Focal Mechanism with GAfocal***

GAfocal uses genetic algorithms to determine earthquake focal mechanisms from polarity data.

In [ ]:
dout_file_name = pol_mag_to_dout(
    ori_dout=h3dd.get_dout(),
    result_path=RESULT_DIR,
    gamma_event=gamma.get_events(),
    polarity_picks=dt_polarity.get_picks(),
    magnitude_events=mag.get_events(),
    magnitude_picks=mag.get_picks()
)

# Run GAfocal genetic algorithm
print("Running genetic algorithm for focal mechanisms...")
gafocal = GAfocal(
    dout_file_name=dout_file_name,
    result_path=RESULT_DIR
)
gafocal.run()

---
### ***Visualization***

Here we simply display the beachball on the map

In [34]:
%matplotlib inline
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import LightSource

import pygmt
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes

from pyrocko import moment_tensor as pmt
from pyrocko.plot import beachball, mpl_color

def check_hms_gafocal(hms: str):
    """
    check whether the gafocal format's second overflow
    """
    minute = int(hms[3:5])
    second = int(hms[6:8])

    if second >= 60:
        minute += second // 60
        second = second % 60

    fixed_hms = hms[:3] + f'{minute:02d}' + hms[5:6] + f'{second:02d}'
    return fixed_hms

def txt_preprocessor(df: pd.DataFrame) -> pd.DataFrame:
    df[1] = df[1].apply(check_hms_gafocal)
    df['time'] = df[0].apply(lambda x: x.replace('/', '-')) + 'T' + df[1]
    for col in [6, 8, 10]:
        df[col] = df[col].map(lambda x: int(x.split('+')[0]))
    df = df.rename(
        columns={
            2: 'longitude',
            3: 'latitude',
            4: 'depth_km',
            5: 'magnitude',
            6: 'strike',
            7: 'strike_err',
            8: 'dip',
            9: 'dip_err',
            10: 'rake',
            11: 'rake_err',
            12: 'quality_index',
            13: 'num_of_polarity',
        }
    )
    mask = [i for i in df.columns.tolist() if isinstance(i, str)]
    df = df[mask]
    return df

def read_gafocal(gafocal_txt: str | Path) -> pd.DataFrame:   
    df = pd.read_csv(
        gafocal_txt,
        sep=r'\s+',
        header=None,
        dtype={0: 'str', 1: 'str'},
    )
    df = txt_preprocessor(df)    
    return df

def get_mapview(
    region: list,
    ax: GeoAxes,
    cmap='gray',
    hillshade=True,
    resolution='15s',
    vert_exag=10,
    ):
    # from matplotlib.colors import ListedColormap
    """## Plotting the basic map.

    Notice here, the ax should be a GeoAxes! A subclass of `matplotlib.axes.Axes`.
    """
    topo = (
        pygmt.datasets.load_earth_relief(resolution=resolution, region=region).to_numpy()
        / 1e3
    )  # km
    x = np.linspace(region[0], region[1], topo.shape[1])
    y = np.linspace(region[2], region[3], topo.shape[0])
    xgrid, ygrid = np.meshgrid(x, y)
    ls = LightSource(azdeg=0, altdeg=45)

    ax.pcolormesh(
        xgrid,
        ygrid,
        ls.hillshade(topo, vert_exag=vert_exag, dx=1, dy=1) if hillshade else topo,
        vmin=-1 if hillshade else None,
        shading='gouraud',
        cmap=cmap,
        alpha=1.0,
        antialiased=True,
        rasterized=True,
    )

    # cartopy setting
    ax.coastlines()
    ax.set_extent(region)

    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = False  # Turn off top labels
    gl.right_labels = False  # Turn off top labels
    gl.xlabel_style = {
        'weight': 'bold',
        'size': 10
    }
    gl.ylabel_style = {
        'rotation': 90,
        'weight': 'bold',
        'size': 10
    }

In [ ]:
df = read_gafocal(RESULT_DIR / "gafocal_results.txt")
df_station = pd.read_csv(STATION)

region = [
    float(df['longitude'].min()) - 0.2,
    float(df['longitude'].max()) + 0.1,
    float(df['latitude'].min()) - 0.1,
    float(df['latitude'].max()) + 0.1
]

fig = plt.figure(figsize=(12, 18))    
geo_ax = fig.add_subplot(projection=ccrs.PlateCarree())
get_mapview(
    region=region,
    ax=geo_ax,
)
geo_ax.set_xlim(region[0], region[1])
geo_ax.set_ylim(region[2], region[3])
# Station
geo_ax.scatter(
    df_station['longitude'],
    df_station['latitude'],
    s=50,
    c='c',
    marker='^',
    edgecolors='k',
    alpha=0.7,
    rasterized=True,
    label='Seismometer',
    zorder=4,
)
# geo_ax.set_aspect('auto')

for i, row in enumerate(df.itertuples()):
    mt = pmt.MomentTensor(
        strike=row.strike,
        dip=row.dip,
        rake=row.rake,
    )
    beachball.plot_beachball_mpl(
        mt, geo_ax, size=20, beachball_type='full',
        position=(row.longitude, row.latitude), linewidth=1,
        color_t=mpl_color('k'),
        zorder=9
    )